In [1]:
import pandas as pd

In [28]:
df = pd.read_csv("trainingmain.csv")

In [29]:
df.head()

,Unnamed: 0,question,contexts,ground_truth,answers
0,0,How can the serum half-life of nanobody candid...,"[' , at the cost of a shortened in vivo half-l...",Linking a nanobody to anti-human serum albumin...,The serum half-life of nanobody candidates can...
1,2,How do functional changes in VSS affect visual...,[' increase in alpha-gamma phase-amplitude cou...,Functional changes in VSS affect visual proces...,Functional changes in Visual Snow Syndrome (VS...
2,4,What is the empirical variability among circul...,[' sequences extracted from GISAID to estimate...,The empirical variability among circulating SA...,The empirical variability among circulating SA...
3,5,What is the role of the antiviral response in ...,['-15 is involved in natural killer cell diffe...,The antiviral response plays a role in the imm...,The antiviral response in the immune system pl...
4,6,What is the appearance of the classic steric m...,[' bond length and angle values on the accessi...,The appearance of the classic steric map deriv...,The appearance of the classic steric map deriv...


In [30]:
df.columns

Index(['Unnamed: 0', 'question', 'contexts', 'ground_truth', 'answers'], dtype='object')

In [31]:
df = df[['question', 'contexts', 'ground_truth', 'answers']]

In [32]:
df.head()

,question,contexts,ground_truth,answers
0,How can the serum half-life of nanobody candid...,"[' , at the cost of a shortened in vivo half-l...",Linking a nanobody to anti-human serum albumin...,The serum half-life of nanobody candidates can...
1,How do functional changes in VSS affect visual...,[' increase in alpha-gamma phase-amplitude cou...,Functional changes in VSS affect visual proces...,Functional changes in Visual Snow Syndrome (VS...
2,What is the empirical variability among circul...,[' sequences extracted from GISAID to estimate...,The empirical variability among circulating SA...,The empirical variability among circulating SA...
3,What is the role of the antiviral response in ...,['-15 is involved in natural killer cell diffe...,The antiviral response plays a role in the imm...,The antiviral response in the immune system pl...
4,What is the appearance of the classic steric m...,[' bond length and angle values on the accessi...,The appearance of the classic steric map deriv...,The appearance of the classic steric map deriv...


In [33]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"Answers": "' + row['answers'] + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['question']},
                {"role": "assistant", "content": json_response},
            ]
        })
    return fine_tuning_data


In [34]:
converted_data = convert_to_gpt35_format(df)

In [35]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    random_state=42  # for reproducibility
)


In [36]:
import json
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)


In [47]:
from openai import OpenAI
client = OpenAI(api_key="")

# Upload Training and Validation Files
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

# Create Fine-Tuning Job
suffix_name = "FineTuning Covid19"
response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)


In [44]:
response

FineTuningJob(id='ftjob-wP6Njro7tMGErH0buGUUp6rb', created_at=1710583315, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-QGMQNqzO5AZpg9xl00oxeaGx', result_files=[], status='validating_files', trained_tokens=None, training_file='file-ymqZfdy9kxlGt8YuXxkVIXwe', validation_file='file-UhdBHRAdYUc5gcp4CbuCQo21', user_provided_suffix='FineTuning Covid19')

In [48]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-ptAeTbZEewoJMu3KJdUCUsOj', created_at=1710584025, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-QGMQNqzO5AZpg9xl00oxeaGx', result_files=[], status='validating_files', trained_tokens=None, training_file='file-AUhAT9xiKcWzVFj7N7niokBP', validation_file='file-g9Nv6I8mSd16WzAZ3P4WbtzL', user_provided_suffix='FineTuning Covid19'), FineTuningJob(id='ftjob-wP6Njro7tMGErH0buGUUp6rb', created_at=1710583315, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-QGMQNqzO5AZpg9xl00oxeaGx', result_files

In [49]:
response = client.fine_tuning.jobs.retrieve("ftjob-wP6Njro7tMGErH0buGUUp6rb")
response

FineTuningJob(id='ftjob-wP6Njro7tMGErH0buGUUp6rb', created_at=1710583315, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-QGMQNqzO5AZpg9xl00oxeaGx', result_files=[], status='running', trained_tokens=None, training_file='file-ymqZfdy9kxlGt8YuXxkVIXwe', validation_file='file-UhdBHRAdYUc5gcp4CbuCQo21', user_provided_suffix='FineTuning Covid19')

In [50]:

fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: None


In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):
    formatted_message = [{"role": "user", "content": row['question']}]
    return formatted_message

def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )
    return response.choices[0].message.content

def store_predictions(test_df, fine_tuned_model_id):
    test_df['Prediction'] = None
    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")


In [53]:
test_df = pd.read_csv("test_df.csv")
store_predictions(test_df, fine_tuned_model_id)

BadRequestError: Error code: 400 - {'error': {'message': 'you must provide a model parameter', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
df = pd.read_csv("train.csv")
df = df.dropna(subset=['ground_truth'])
df = df[['question', 'contexts', 'ground_truth']]

test_questions = df["question"].values.tolist()
test_groundtruths = df["ground_truth"].values.tolist()

answers = []
contexts = []

for question in test_questions:
  response = qa.invoke({"query" : question})
  answers.append(response["result"])
  contexts.append([context.page_content for context in response['source_documents']])


In [ ]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})
response_dataset[0]